## Limpieza de Prime Video

### Pedimos los datos a la API

In [1]:
import numpy as np
import pandas as pd
import requests as rq

In [2]:
url = 'https://api.watchmode.com/v1/'
API_KEY = 'AvZTb8hmuinRzMat9WdZUa9Iy2DxrPxkK3boMnwg'
region = "CL"
plataformas = {
    'Netflix': 203,
    'Hulu': 157,
    'Max': 387,
    'Prime': 26,
    'Disney+': 372,
    'Paramount+': 444
}

In [3]:
# FUNCION
def buscar_peliculas(plataforma):
    ## Parametros del GET
    parametros = {
        'apiKey': API_KEY,
        'types': 'movie',
        # 'regions': region,  # Por alguna razon la request no funciona al especificar la region asi que queda comentada
        'source_ids': plataformas[plataforma]
    }
    url_get = url + 'list-titles/'

    ## Primer get
    response = rq.get(url_get, params=parametros)
    json_1 = response.json()
    
    print(response, json_1['total_pages'], json_1['total_results'])
    
    paginas_totales = json_1['total_pages']
    df_1 = pd.DataFrame(json_1['titles'])
    resultados_totales = json_1['total_results']

    # Ciclo
    dataframes = []
    dataframes.append(df_1)

    for i in range(2, paginas_totales + 1):
        parametros['page'] = i
        response = rq.get(url_get, params=parametros)
        json_n = response.json()
        df_n = pd.DataFrame(json_n['titles'])
        dataframes.append(df_n)

    # Se juntan todos los DataFrames
    dataframe_final = pd.concat(dataframes, ignore_index=True)
    return (dataframe_final, resultados_totales)
    return response

### Pedimos los datos

In [7]:
df_hbo_1 = buscar_peliculas("Max")

<Response [200]> 5 1182


Verificamos que no falten de datos:

In [8]:
largo_dataframe = len(df_hbo_1[0])
largo_segun_API = df_hbo[1]
print([largo_dataframe, largo_segun_API])

[1182, 1182]


Vemos que no hubo perdida de datos, por lo que están todos

In [9]:
df_hbo_2 = df_hbo_1[0]
df_hbo_2

,id,title,year,imdb_id,tmdb_id,tmdb_type,type
0,1703330,Superman,2025,tt5950044,1061474,movie,movie
1,1783110,Sinners,2025,tt31193180,1233413,movie,movie
2,1532981,The Batman,2022,tt1877830,414906,movie,movie
3,1718402,Heretic,2024,tt28015403,1138194,movie,movie
4,1653388,Mickey 17,2025,tt12299608,696506,movie,movie
...,...,...,...,...,...,...,...
1177,1307384,Presenting Princess Shaw,2016,tt4161074,357799,movie,movie
1178,1613151,Stockton on My Mind,2020,tt11905696,712393,movie,movie
1179,1628337,Lily Topples The World,2021,tt13963956,795625,movie,movie
1180,176887,Citizen USA: A 50 State Road Trip,2011,tt2009450,82325,movie,movie


In [10]:
df_hbo_2["type"].unique()

array(['movie'], dtype=object)

In [11]:
df_hbo_2["tmdb_type"].unique()

array(['movie'], dtype=object)

Como son la misma columna entre ellas dos y vemos que no hay ningun dato colado, las eliminamos porque no son necesarias

In [12]:
df_hbo_3 = df_hbo_2[["id",'title', 'year', 'imdb_id', 'tmdb_id']]

### Valores nulos


In [13]:
df_hbo_3["id"].isna().unique()

array([False])

In [14]:
df_hbo_3["title"].isna().unique()

array([False])

In [15]:
df_hbo_3["year"].isna().unique()

array([False])

In [16]:
df_hbo_3["imdb_id"].isna().unique()

array([False])

Ahora vemos si esque existen duplicados

In [17]:
df_hbo_3.duplicated().sum()

np.int64(0)

No se encontraron duplicados en el DataFrame

### Vemos los valores unicos
Ya que para cada pelicula deberían existir un unico id

In [18]:
df_hbo_3["id"].value_counts()

id
1952076    1
1703330    1
1783110    1
1532981    1
1718402    1
          ..
1129886    1
1193690    1
1295258    1
1496273    1
1529898    1
Name: count, Length: 1182, dtype: int64

Vemos si hay repetición en los titulos

In [20]:
df_hbo_3["title"].value_counts()

title
Mortal Kombat                               2
Red Dawn                                    2
Superman                                    2
A Nightmare on Elm Street                   2
Dr. Jekyll and Mr. Hyde                     2
                                           ..
Westfront 1918                              1
537 Votes                                   1
Nightingale                                 1
If You're Not in the Obit, Eat Breakfast    1
Bloody Trophy                               1
Name: count, Length: 1171, dtype: int64

Revisamos a que se debe la repetición de los títulos

In [22]:
df_hbo_3[df_hbo_3["title"] == "Mortal Kombat"]

,id,title,year,imdb_id,tmdb_id
41,1559743,Mortal Kombat,2021,tt0293429,460465
273,1260826,Mortal Kombat,1995,tt0113855,9312


Podemos ver que se debe a los diferentes años en los cuales salío la pelicula

### Revisamos los años de estreno de las películas

In [23]:
df_hbo_3["year"].value_counts()

year
2024    70
2022    57
2016    51
2021    43
2023    38
        ..
1923     2
1921     1
1935     1
1929     1
1937     1
Name: count, Length: 103, dtype: int64

Vemos que esta todo bien en los rangos de año

### Ahora revisamos si un imbd_id sale repetido

In [25]:
df_hbo_3["imdb_id"].value_counts()

imdb_id
tt28769082    1
tt5950044     1
tt31193180    1
tt1877830     1
tt28015403    1
             ..
tt4123432     1
tt7286456     1
tt6751668     1
tt1160419     1
tt8999762     1
Name: count, Length: 1182, dtype: int64

Vemos que esta todo bien

In [26]:
df_hbo_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1182 entries, 0 to 1181
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       1182 non-null   int64 
 1   title    1182 non-null   object
 2   year     1182 non-null   int64 
 3   imdb_id  1182 non-null   object
 4   tmdb_id  1182 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 46.3+ KB


In [27]:
df_hbo_3["year"].describe()

count    1182.000000
mean     1992.780880
std        29.845896
min      1921.000000
25%      1966.000000
50%      2006.000000
75%      2019.000000
max      2025.000000
Name: year, dtype: float64

### Lo pasamos a .csv

In [28]:
df_hbo_3.to_csv("api_max.csv", index=False)